In [7]:
# importando libs
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import os
from datetime import datetime
from datetime import timedelta
import yfinance as yf
import pickle
from prophet import Prophet
from prophet.serialize import model_to_json, model_from_json
from prophet import plot
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [4]:
date_today = datetime.today().strftime("%Y-%m-%d")
data_init = "2014-01-01"

df_btc = yf.download("BTC-USD", data_init, date_today)

df_btc.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100


In [5]:
df_btc.reset_index(inplace=True)
df_btc.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
2916,2022-09-11,21678.542969,21770.552734,21406.945312,21769.255859,21769.255859,34493951963
2917,2022-09-12,21770.148438,22439.181641,21603.896484,22370.449219,22370.449219,50212088965
2918,2022-09-13,22371.480469,22673.820312,20062.669922,20296.707031,20296.707031,51091116622
2919,2022-09-14,20184.554688,20467.201172,19793.396484,20241.089844,20241.089844,37872380889
2920,2022-09-15,20242.289062,20318.166016,19636.734375,19701.210938,19701.210938,36389011503


In [8]:
df = df_btc[["Date", "Adj Close"]]
df.rename(columns= { "Date": "ds", "Adj Close": "y" }, inplace=True)

In [9]:
df.tail()

,ds,y
2916,2022-09-11,21769.255859
2917,2022-09-12,22370.449219
2918,2022-09-13,20296.707031
2919,2022-09-14,20241.089844
2920,2022-09-15,19701.210938


In [10]:
df.isnull().sum().sum()
df['ds'].isnull().sum()
df['y'].isnull().sum()

0

In [11]:
df.dtypes

ds    datetime64[ns]
y            float64
dtype: object

In [12]:
model = Prophet(seasonality_mode="multiplicative")
model.fit(df)

23:25:58 - cmdstanpy - INFO - Chain [1] start processing
23:26:01 - cmdstanpy - INFO - Chain [1] done processing


In [13]:
df_future = model.make_future_dataframe(periods=7)
df_future.tail()

,ds
2923,2022-09-18
2924,2022-09-19
2925,2022-09-20
2926,2022-09-21
2927,2022-09-22


In [14]:
# predict 
predict = model.predict(df_future)
predict

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2014-09-17,154.028044,-3412.416425,3640.319542,154.028044,154.028044,-0.053055,-0.053055,-0.053055,0.001817,0.001817,0.001817,-0.054872,-0.054872,-0.054872,0.0,0.0,0.0,145.856100
1,2014-09-18,154.554006,-3183.485302,3624.878759,154.554006,154.554006,-0.053541,-0.053541,-0.053541,-0.002647,-0.002647,-0.002647,-0.050894,-0.050894,-0.050894,0.0,0.0,0.0,146.279074
2,2014-09-19,155.079967,-3134.379511,3563.156364,155.079967,155.079967,-0.046410,-0.046410,-0.046410,-0.000077,-0.000077,-0.000077,-0.046333,-0.046333,-0.046333,0.0,0.0,0.0,147.882708
3,2014-09-20,155.605929,-3226.277918,3600.727874,155.605929,155.605929,-0.040539,-0.040539,-0.040539,0.000618,0.000618,0.000618,-0.041157,-0.041157,-0.041157,0.0,0.0,0.0,149.297806
4,2014-09-21,156.131890,-3451.690868,3442.747008,156.131890,156.131890,-0.034724,-0.034724,-0.034724,0.000609,0.000609,0.000609,-0.035334,-0.035334,-0.035334,0.0,0.0,0.0,150.710353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2923,2022-09-18,31367.840148,26262.010363,33256.018015,31367.840148,31367.840148,-0.050284,-0.050284,-0.050284,0.000609,0.000609,0.000609,-0.050894,-0.050894,-0.050894,0.0,0.0,0.0,29790.525141
2924,2022-09-19,31331.493892,26445.367281,33187.226154,31331.493892,31331.493892,-0.045481,-0.045481,-0.045481,0.000852,0.000852,0.000852,-0.046333,-0.046333,-0.046333,0.0,0.0,0.0,29906.512581
2925,2022-09-20,31295.147637,26447.052021,33274.360654,31295.147637,31295.147637,-0.042329,-0.042329,-0.042329,-0.001172,-0.001172,-0.001172,-0.041157,-0.041157,-0.041157,0.0,0.0,0.0,29970.454149
2926,2022-09-21,31258.801381,26864.355945,33728.451212,31258.801381,31258.801381,-0.033517,-0.033517,-0.033517,0.001817,0.001817,0.001817,-0.035334,-0.035334,-0.035334,0.0,0.0,0.0,30211.105462


In [15]:
predict[["ds", "yhat_lower", "yhat_upper", "yhat"]].tail()

,ds,yhat_lower,yhat_upper,yhat
2923,2022-09-18,26262.010363,33256.018015,29790.525141
2924,2022-09-19,26445.367281,33187.226154,29906.512581
2925,2022-09-20,26447.052021,33274.360654,29970.454149
2926,2022-09-21,26864.355945,33728.451212,30211.105462
2927,2022-09-22,26882.914690,33634.452455,30239.568597


In [16]:
# grafico 
#plot.plot(model, predict)

In [19]:
# Salvar modelo
with open('modelo_btc_usd.pckl', 'wb') as fout:
    pickle.dump(model, fout)